# Meta Models - bprMF

In this notebook we prototype a set of Meta Models. In particular, we use the s-value trained using the bprMF algorithm. We then use these as input features to our model. These are combined with book level features in the hopes to enhance the predictive accuracy of the model.

In [1]:
import json
import os
import random
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import scipy.sparse as sp

random.seed(42)
np.random.seed(42)

### Loading S-Value

We load the S-Value from the bprMF model.

In [2]:
MAPPING_DIR = '../mappings/'

s_values_df = pd.read_csv(MAPPING_DIR+"goodreads_s_values_bpr.csv")

In [3]:
cols_to_keep = ['user_number', 'item_number', 's']

s_values_df = s_values_df[cols_to_keep]

In [4]:
s_values_df['user_number'] = s_values_df['user_number'].apply(lambda x: str(x))
s_values_df['item_number'] = s_values_df['item_number'].apply(lambda x: str(x))
s_values_df['user_item_id'] = s_values_df['user_number'] + "-" + s_values_df['item_number']

### Load Data

In [5]:
OUTPUT_DATA_DIR = "../output_data/"

train_df_processed = pd.read_csv(OUTPUT_DATA_DIR+"text_processed_training.csv")
val_df_processed = pd.read_csv(OUTPUT_DATA_DIR+"text_processed_validation.csv")
test_df_processed = pd.read_csv(OUTPUT_DATA_DIR+"text_processed_testing.csv")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Combining Features

We now combine the features engineered on the raw data with the trained s values.

Both datasets have interactions for combinations of user ids and book ids. However, the chainRec model expects the ids to be numberic. So we apply this same mapping to the raw data and then join the two datasets together to get our augmented set of features.

In [6]:
def load_mapping(mapping_file):
    """Loads the mapping from `mapping_file`.
    
    Parameters
    ----------
    mapping_file: str
        The name of the mapping file to import.
    
    Returns
    -------
    pd.DataFrame
        The DataFrame created from the mapping.
    
    """
    return pd.read_csv(os.path.join("../mappings", "{}.csv".format(mapping_file)))

In [7]:
user_map = load_mapping("user_map")
book_map = load_mapping("book_map")

In [8]:
book_map['book_id'] = book_map['book_id'].apply(lambda x: str(x))

In [9]:
def create_user_item_id(data_df, u_map, i_map):
    """Creates a user-item ID for the records in `data_df`.

    The user-item ID is created from `u_map` and `i_map`.
    Both mappings, map text user IDs to numeric user IDs
    and these numeric user IDs are combined to form the
    user-item ID.

    Parameters
    ----------
    data_df: pd.DataFrame
        The DataFrame for which the user-item ID is created.
    u_map: pd.DataFrame
        A DataFrame containing a mapping from a text user ID to
        a numeric user ID.
    i_map: pd.DataFrame
        A DataFrame containing a mapping from a text item ID to
        a numeric item ID.

    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from `data_df` after adding a
        user-item ID field based on `u_map` and `i_map`.

    """
    data_df['book_id'] = data_df['book_id'].apply(lambda x: str(x))
    data_df = pd.merge(data_df, u_map, how="left", on=["user_id"])
    data_df = pd.merge(data_df, i_map, how="left", on=["book_id"])
    data_df['user_number'] = data_df['user_number'].apply(lambda x: str(x))
    data_df['book_number'] = data_df['book_number'].apply(lambda x: str(x))
    data_df['user_item_id'] = data_df['user_number'] + "-" + data_df['book_number']
    return data_df.drop(columns=['user_number', 'book_number'])

In [10]:
train_df = create_user_item_id(train_df_processed, user_map, book_map)
val_df = create_user_item_id(val_df_processed, user_map, book_map)
test_df = create_user_item_id(test_df_processed, user_map, book_map)

In [11]:
s_values_df.drop(columns=['user_number', 'item_number'], inplace=True)

In [12]:
train_df_s = pd.merge(train_df, s_values_df, how='left', on=['user_item_id'])
val_df_s = pd.merge(val_df, s_values_df, how='left', on=['user_item_id'])
test_df_s = pd.merge(test_df, s_values_df, how='left', on=['user_item_id'])

In [13]:
train_df_s.drop(columns=['user_item_id'], inplace=True)
val_df_s.drop(columns=['user_item_id'], inplace=True)
test_df_s.drop(columns=['user_item_id'], inplace=True)

### Saving Augmented Data

We save the augmented train, test, and validation datasets for easier access on future models

In [ ]:
train_df_s.to_csv(OUTPUT_DATA_DIR+"training_s_vals_bpr.csv", index=False)
val_df_s.to_csv(OUTPUT_DATA_DIR+"validation_s_vals_bpr.csv", index=False)
test_df_s.to_csv(OUTPUT_DATA_DIR+"testing_s_vals_bpr.csv", index=False)

In [14]:
pd.set_option('display.max_columns', None)

### Non Language Features

We select the set of non language features which will be combined with the vectorized language features to get a full set of model features.

We also scale the count features. As counts tend to grow exponentially we use a log transform to make the counts linear.

In [16]:
feature_columns = ['text_reviews_count', 'is_ebook', 'average_rating', 'num_pages',
                   'ratings_count', 'is_translated', 'is_in_series', 'series_length', 
                   'is_paperback', 'is_hardcover', 'is_audio', 'from_penguin', 
                   'from_harpercollins', 'from_university_press', 'from_vintage',
                   'from_createspace', 'publication_year', 'author_a', 'author_b', 
                   'author_c', 'author_d', 'author_e', 'author_f', 
                   'book_shelved_count', 'shelved_count', 'read_count', 'rated_count', 
                   'recommended_count', 'title_len', 's']
X_train_reg = train_df_s[feature_columns]
X_val_reg = val_df_s[feature_columns]
X_test_reg = test_df_s[feature_columns]

In [17]:
def log_transform_columns(data_df, cols):
    """Applies a log transform to `cols` in `data_df`.

    Parameters
    ----------
    data_df: pd.DataFrame
        The DataFrame in which the columns will be transformed.
    cols: collection
        The columns in `data_df` to be log scaled.

    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from `data_df` after log scaling
        the columns `cols`.

    """
    for col in cols:
        data_df[col] = data_df[col].apply(lambda x: np.log(x) if x > 0 else 0)
    return data_df

In [18]:
log_transform_cols = ['text_reviews_count', 'ratings_count', 'shelved_count', 'read_count', 'rated_count', 'recommended_count']
X_train_reg = log_transform_columns(X_train_reg, log_transform_cols)
X_val_reg = log_transform_columns(X_val_reg, log_transform_cols)
X_test_reg = log_transform_columns(X_test_reg, log_transform_cols)

<ipython-input-17-49ee2668773d>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[col] = data_df[col].apply(lambda x: np.log(x) if x > 0 else 0)


### Non-Language Models

We try some models that do not use any text based features first. Just by augmenting with the s values.

##### Unscaled S Values

First we use the raw s values

In [19]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()

X_train_reg1 = min_max_scaler.fit_transform(X_train_reg)
X_val_reg1 = min_max_scaler.transform(X_val_reg)
X_test_reg1 = min_max_scaler.transform(X_test_reg)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

reg_model = LogisticRegression(max_iter=200)
reg_model.fit(X_train_reg1, train_df_s['recommended'])

train_AUC = roc_auc_score(train_df_s['recommended'], reg_model.predict(X_train_reg1))
val_AUC = roc_auc_score(val_df_s['recommended'], reg_model.predict(X_val_reg1))

print("Training AUC: {}".format(train_AUC))
print("Validation AUC: {}".format(val_AUC))

Training AUC: 0.7439334653401747
Validation AUC: 0.6360430291499664


In [21]:
reg_df = pd.DataFrame({'feature': feature_columns,
                       'regression_coefficient': reg_model.coef_[0]})

reg_df.head(len(feature_columns))

,feature,regression_coefficient
0,text_reviews_count,-1.462669
1,is_ebook,0.333037
2,average_rating,1.141813
3,num_pages,-0.995051
4,ratings_count,-3.600942
5,is_translated,-0.109992
6,is_in_series,-0.113685
7,series_length,0.217881
8,is_paperback,-0.065874
9,is_hardcover,0.022676


In [22]:
from xgboost import XGBClassifier

xg_cls = XGBClassifier(
    objective='binary:logistic', learning_rate=0.1,
    max_depth=1, n_estimators=1000)

xg_cls.fit(X_train_reg1, train_df_processed['recommended'])
train_AUC = roc_auc_score(
    train_df_processed['recommended'], xg_cls.predict(X_train_reg1))
val_AUC = roc_auc_score(
    val_df_processed['recommended'], xg_cls.predict(X_val_reg1))

print("Training AUC: {}".format(train_AUC))
print("Validation AUC: {}".format(val_AUC))

Training AUC: 0.7516202741272164
Validation AUC: 0.6406109622199797


In [23]:
from sklearn.ensemble import RandomForestClassifier

ranfor_model = RandomForestClassifier(n_estimators=1000, max_depth=15)
ranfor_model.fit(X_train_reg1, train_df_processed['recommended'])

train_AUC = roc_auc_score(
    train_df_processed['recommended'], ranfor_model.predict(X_train_reg1))
val_AUC = roc_auc_score(
    val_df_processed['recommended'], ranfor_model.predict(X_val_reg1))

print("Training AUC: {}".format(train_AUC))
print("Validation AUC: {}".format(val_AUC))

Training AUC: 0.7948962957466644
Validation AUC: 0.6413121497698786


##### Scaled S Values

Next we try the models after scaling the s values using a sigmoid function

In [24]:
def sigmoid(val):
    """Applies the sigmoid function to `val`.
    
    The sigmoid function has the form
    f(x) = 1 / (1 + exp(-x))
    
    Parameters
    ----------
    val: float
        The operand to the sigmoid function.
    
    Returns
    -------
    float
        The result of applying the sigmoid
        function to `val`.
    
    """
    return 1 / (1 + np.exp(-val))

In [25]:
def scale_s_values(data_df):
    """Applies the sigmoid function to the s values in `data_df`.
    
    Parameters
    ---------
    data_df: pd.DataFrame
        The DataFrame for which the operation is performed.
    
    Returns
    -------
    pd.DataFrame
        The DataFrame that results from scaling the s values
        in `data_df`.
    
    """
    for s_col in ["s"]:
        data_df[s_col] = data_df[s_col].apply(lambda x: sigmoid(x))
    return data_df

In [26]:
min_max_scaler = MinMaxScaler()

X_train_reg2 = min_max_scaler.fit_transform(scale_s_values(X_train_reg))
X_val_reg2 = min_max_scaler.transform(scale_s_values(X_val_reg))
X_test_reg2 = min_max_scaler.transform(scale_s_values(X_test_reg))

<ipython-input-25-84ed0bac13a2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[s_col] = data_df[s_col].apply(lambda x: sigmoid(x))
<ipython-input-25-84ed0bac13a2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[s_col] = data_df[s_col].apply(lambda x: sigmoid(x))
<ipython-input-25-84ed0bac13a2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [27]:
reg_model = LogisticRegression(max_iter=1000)
reg_model.fit(X_train_reg2, train_df_s['recommended'])

train_AUC = roc_auc_score(train_df_s['recommended'], reg_model.predict(X_train_reg2))
val_AUC = roc_auc_score(val_df_s['recommended'], reg_model.predict(X_val_reg2))

print("Training AUC: {}".format(train_AUC))
print("Validation AUC: {}".format(val_AUC))

Training AUC: 0.7400259390940621
Validation AUC: 0.6355098495680449


In [28]:
reg_df = pd.DataFrame({'feature': feature_columns,
                       'regression_coefficient': reg_model.coef_[0]})

reg_df.head(len(feature_columns))

,feature,regression_coefficient
0,text_reviews_count,-0.721495
1,is_ebook,0.371479
2,average_rating,1.830329
3,num_pages,-0.865162
4,ratings_count,1.687757
5,is_translated,-0.181481
6,is_in_series,-0.366161
7,series_length,0.795871
8,is_paperback,-0.054652
9,is_hardcover,-0.013623


In [29]:
X_train_reg2 = scale_s_values(X_train_reg)
X_val_reg2 = scale_s_values(X_val_reg)
X_test_reg2 = scale_s_values(X_test_reg)

<ipython-input-25-84ed0bac13a2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[s_col] = data_df[s_col].apply(lambda x: sigmoid(x))


In [30]:
from xgboost import XGBClassifier

xg_cls = XGBClassifier(
    objective='binary:logistic', learning_rate=0.1,
    max_depth=1, n_estimators=1000)

xg_cls.fit(X_train_reg2, train_df_processed['recommended'])
train_AUC = roc_auc_score(
    train_df_processed['recommended'], xg_cls.predict(X_train_reg2))
val_AUC = roc_auc_score(
    val_df_processed['recommended'], xg_cls.predict(X_val_reg2))

print("Training AUC: {}".format(train_AUC))
print("Validation AUC: {}".format(val_AUC))

Training AUC: 0.7518654498676378
Validation AUC: 0.6415307222710519


In [31]:
from sklearn.ensemble import RandomForestClassifier

ranfor_model = RandomForestClassifier(n_estimators=1000, max_depth=15)
ranfor_model.fit(X_train_reg2, train_df_processed['recommended'])

train_AUC = roc_auc_score(
    train_df_processed['recommended'], ranfor_model.predict(X_train_reg2))
val_AUC = roc_auc_score(
    val_df_processed['recommended'], ranfor_model.predict(X_val_reg2))

print("Training AUC: {}".format(train_AUC))
print("Validation AUC: {}".format(val_AUC))

Training AUC: 0.7892983862945882
Validation AUC: 0.6479742905838697


### Word2Vec

We create vector embeddings of the words in the book descriptions. Word2Vec captures the most important words and then the vectors for each important word in the book description are averaged to get a vector for the book

In [32]:
book_df = train_df_s[['book_id', 'cleaned_text']]
book_df = book_df.drop_duplicates(subset=['book_id'])

book_df['cleaned_text'] = book_df['cleaned_text'].apply(lambda x: "" if pd.isnull(x) else x)

w2v = Word2Vec(list(book_df['cleaned_text']), size=200, window=10, min_count=1)

In [33]:
def create_book_vector(book_text, vec_length):
    """Creates a vector for the book given by `book_text`.

    The word vectors for each word in `book_text` are
    averaged to build a vector for the book.

    Parameters
    ----------
    book_text: str
        The book text for which the vector is generated.

    Returns
    -------
    vector
        A vector for the book.

    """
    text_vecs = [word for word in str(book_text) if word in w2v.wv.vocab]
    if len(text_vecs) > 0:
        return np.mean(w2v[text_vecs], axis=0)
    return np.zeros(vec_length)

In [34]:
train_df_s['book_vector'] = train_df_s['cleaned_text'].apply(lambda x: create_book_vector(x, 200))
val_df_s['book_vector'] = val_df_s['cleaned_text'].apply(lambda x: create_book_vector(x, 200))
test_df_s['book_vector'] = test_df_s['cleaned_text'].apply(lambda x: create_book_vector(x, 200))

<ipython-input-33-4533799a3578>:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return np.mean(w2v[text_vecs], axis=0)


In [35]:
def create_book_vec_df(book_vecs, indices):
    """Creates a dataframe from `book_vecs`.

    Each numpy array in `book_vecs` is converted to a
    row in the resulting dataframe.

    Parameters
    ----------
    book_vecs: list
        A list of numpy arrays where each array corresponds
        to the book vector for a book.
    indicies: np.array
        A numpy array of indices for the DataFrame

    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from converting `review_vecs`
        to a dataframe.

    """
    book_vec_df = pd.DataFrame(np.vstack(book_vecs))
    book_vec_df.columns = ["word" + str(col) for col in book_vec_df.columns]
    book_vec_df.index = indices
    return book_vec_df

In [36]:
train_wv = create_book_vec_df(train_df_s['book_vector'], train_df_s.index)
val_wv = create_book_vec_df(val_df_s['book_vector'], val_df_s.index)
test_wv = create_book_vec_df(test_df_s['book_vector'], test_df_s.index)

In [37]:
min_max_scaler = MinMaxScaler()

X_train_reg2 = min_max_scaler.fit_transform(scale_s_values(X_train_reg))
X_val_reg2 = min_max_scaler.transform(scale_s_values(X_val_reg))
X_test_reg2 = min_max_scaler.transform(scale_s_values(X_test_reg))

<ipython-input-25-84ed0bac13a2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[s_col] = data_df[s_col].apply(lambda x: sigmoid(x))
<ipython-input-25-84ed0bac13a2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[s_col] = data_df[s_col].apply(lambda x: sigmoid(x))
<ipython-input-25-84ed0bac13a2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [42]:
X_train_wv_reg = pd.concat([train_wv, pd.DataFrame(np.vstack(X_train_reg2))], axis=1)
X_val_wv_reg = pd.concat([val_wv, pd.DataFrame(np.vstack(X_val_reg2))], axis=1)
X_test_wv_reg = pd.concat([test_wv, pd.DataFrame(np.vstack(X_test_reg2))], axis=1)

In [43]:
reg_model = LogisticRegression(max_iter=1000)
reg_model.fit(X_train_wv_reg, train_df_s['recommended'])

train_AUC = roc_auc_score(train_df_s['recommended'], reg_model.predict(X_train_wv_reg))
val_AUC = roc_auc_score(val_df_s['recommended'], reg_model.predict(X_val_wv_reg))

print("Training AUC: {}".format(train_AUC))
print("Validation AUC: {}".format(val_AUC))

Training AUC: 0.7404464560050454
Validation AUC: 0.635502616531098


In [44]:
X_train_reg_s = scale_s_values(X_train_reg)
X_val_reg_s = scale_s_values(X_val_reg)
X_test_reg_s = scale_s_values(X_test_reg)

X_train_wv_reg1 = pd.concat([train_wv, X_train_reg_s], axis=1)
X_val_wv_reg1 = pd.concat([val_wv, X_val_reg_s], axis=1)
X_test_wv_reg1 = pd.concat([test_wv, X_test_reg_s], axis=1)

<ipython-input-25-84ed0bac13a2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[s_col] = data_df[s_col].apply(lambda x: sigmoid(x))


In [45]:
xg_cls = XGBClassifier(
    objective='binary:logistic', learning_rate=0.1,
    max_depth=2, n_estimators=5000)

xg_cls.fit(X_train_wv_reg1, train_df_processed['recommended'])
train_AUC = roc_auc_score(
    train_df_processed['recommended'], xg_cls.predict(X_train_wv_reg1))
val_AUC = roc_auc_score(
    val_df_processed['recommended'], xg_cls.predict(X_val_wv_reg1))

print("Training AUC: {}".format(train_AUC))
print("Validation AUC: {}".format(val_AUC))

Training AUC: 0.7687408281905521
Validation AUC: 0.6552053391129301


In [46]:
from sklearn.ensemble import RandomForestClassifier

ranfor_model = RandomForestClassifier(n_estimators=1000, max_depth=15)
ranfor_model.fit(X_train_wv_reg1, train_df_processed['recommended'])

train_AUC = roc_auc_score(
    train_df_processed['recommended'], ranfor_model.predict(X_train_wv_reg1))
val_AUC = roc_auc_score(
    val_df_processed['recommended'], ranfor_model.predict(X_val_wv_reg1))

print("Training AUC: {}".format(train_AUC))
print("Validation AUC: {}".format(val_AUC))

Training AUC: 0.7908246271565165
Validation AUC: 0.637264628370165
